In [1]:
# Importar módulos necesarios
import sys
import os
import json
from pathlib import Path
import pandas as pd
import plotly.express as px

# Determinar el directorio raíz y construir la ruta al directorio 'Modulos'
root_dir = Path().resolve()
analisis_segment = os.path.join('ADRpy', 'analisis')
if str(root_dir).endswith(analisis_segment):
    # Notebook ejecutado desde ADRpy/analisis
    modulos_path = root_dir / 'Modulos'
else:
    # Notebook ejecutado desde la raíz del proyecto
    modulos_path = root_dir / 'ADRpy' / 'analisis' / 'Modulos'

# Añadir 'Modulos' al sys.path si existe
if modulos_path.is_dir():
    modulos_path_str = str(modulos_path)
    if modulos_path_str not in sys.path:
        sys.path.append(modulos_path_str)
        print(f"✅ Módulo encontrado y añadido a sys.path: {modulos_path_str}")
else:
    print(f"⚠️ Directorio Modulos no encontrado en: {modulos_path}")

# Probar la carga de datos (sin ejecutar la app completa)
try:
    from Analisis_modelos.data_loader import load_models_data, extract_unique_values
    
    # Ruta al archivo JSON
    json_path = os.path.join('Results', 'modelos_completos_por_celda.json')
    
    if os.path.exists(json_path):
        print("✅ Archivo JSON encontrado")
        
        # Cargar datos
        modelos_por_celda, detalles_por_celda = load_models_data(json_path)
        
        print(f"📊 Modelos por celda: {len(modelos_por_celda)} celdas")
        print(f"📋 Detalles por celda: {len(detalles_por_celda)} celdas")
        
        # Extraer valores únicos para filtros
        unique_values = extract_unique_values(modelos_por_celda)
        
        print("\n🔍 Valores únicos disponibles:")
        for key, values in unique_values.items():
            print(f"  {key}: {len(values)} opciones")
            if len(values) <= 10:  # Mostrar si hay pocas opciones
                print(f"    {values}")
            else:
                print(f"    {values[:5]}... (y {len(values)-5} más)")
        
    else:
        print(f"❌ No se encontró el archivo JSON en: {json_path}")
        print("Verifique que la ruta sea correcta.")
        
except ImportError as e:
    print(f"⚠️ Error de importación: {e}")
    print("Instale las dependencias necesarias: pip install dash plotly pandas")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Módulo encontrado y añadido a sys.path: C:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Modulos
❌ Error: invalid syntax (main_visualizacion_modelos.py, line 251)


In [2]:
# Ejemplo de uso de filtros (sin ejecutar la aplicación completa)
try:
    from Analisis_modelos.data_loader import filter_models, get_parametros_for_aeronave
    
    if 'modelos_por_celda' in locals():
        # Ejemplo 1: Filtrar por aeronave específica
        if unique_values['aeronaves']:
            aeronave_ejemplo = unique_values['aeronaves'][0]
            print(f"🔍 Explorando aeronave: {aeronave_ejemplo}")
            
            # Obtener parámetros para esta aeronave
            parametros_aeronave = get_parametros_for_aeronave(modelos_por_celda, aeronave_ejemplo)
            print(f"   Parámetros disponibles: {parametros_aeronave}")
            
            if parametros_aeronave:
                parametro_ejemplo = parametros_aeronave[0]
                print(f"   Analizando parámetro: {parametro_ejemplo}")
                
                # Filtrar modelos para esta combinación
                modelos_filtrados = filter_models(
                    modelos_por_celda,
                    aeronave=aeronave_ejemplo,
                    parametro=parametro_ejemplo
                )
                
                celda_key = f"{aeronave_ejemplo}|{parametro_ejemplo}"
                if celda_key in modelos_filtrados:
                    modelos = modelos_filtrados[celda_key]
                    print(f"   📊 {len(modelos)} modelos encontrados")
                    
                    # Mostrar resumen de los primeros modelos
                    for i, modelo in enumerate(modelos[:3]):
                        if isinstance(modelo, dict):
                            tipo = modelo.get('tipo', 'N/A')
                            predictores = modelo.get('predictores', [])
                            mape = modelo.get('mape', 0)
                            r2 = modelo.get('r2', 0)
                            
                            print(f"     Modelo {i+1}: {tipo}")
                            print(f"       - Predictores: {', '.join(predictores)}")
                            print(f"       - MAPE: {mape:.3f}%, R²: {r2:.3f}")
        
        # Ejemplo 2: Filtrar por tipo de modelo
        print(f"\n🔍 Filtros por tipo de modelo:")
        tipos_disponibles = unique_values['tipos_modelo'][:3]  # Primeros 3 tipos
        
        modelos_por_tipo = filter_models(
            modelos_por_celda,
            tipos_modelo=tipos_disponibles
        )
        
        print(f"   Modelos filtrados por tipos {tipos_disponibles}: {len(modelos_por_tipo)} celdas")
        
    else:
        print("⚠️ No se han cargado los datos. Ejecute la celda anterior primero.")
        
except ImportError:
    print("⚠️ Módulo no disponible. Instale las dependencias necesarias.")
except Exception as e:
    print(f"❌ Error en filtros: {e}")

❌ Error en filtros: invalid syntax (main_visualizacion_modelos.py, line 251)


In [3]:
# Probar las nuevas funciones para preparar datos de UI
try:
    from Analisis_modelos.plot_model_curves import (
        prepare_normalized_model_curves_data,
        prepare_imputed_points_data,
        filter_single_predictor_models
    )
    
    if 'modelos_por_celda' in locals() and unique_values['aeronaves']:
        # Usar datos del ejemplo anterior
        aeronave_ejemplo = unique_values['aeronaves'][0]
        parametros_aeronave = get_parametros_for_aeronave(modelos_por_celda, aeronave_ejemplo)
        
        if parametros_aeronave:
            parametro_ejemplo = parametros_aeronave[0]
            celda_key = f"{aeronave_ejemplo}|{parametro_ejemplo}"
            
            modelos_filtrados = filter_models(
                modelos_por_celda,
                aeronave=aeronave_ejemplo,
                parametro=parametro_ejemplo
            )
            
            if celda_key in modelos_filtrados:
                modelos = modelos_filtrados[celda_key]
                
                print(f"\n🧪 Probando nuevas funciones para UI:")
                print(f"   Aeronave: {aeronave_ejemplo}")
                print(f"   Parámetro: {parametro_ejemplo}")
                print(f"   Total modelos: {len(modelos)}")
                
                # 1. Filtrar modelos de 1 predictor
                modelos_1pred = filter_single_predictor_models(modelos)
                print(f"   Modelos de 1 predictor: {len(modelos_1pred)}")
                
                if modelos_1pred:
                    # 2. Preparar datos de curvas
                    curves_data = prepare_normalized_model_curves_data(
                        modelos_1pred, 
                        parametro_ejemplo,
                        include_synthetic=True
                    )
                    print(f"   Curvas preparadas: {len(curves_data)}")
                    
                    # Mostrar ejemplo de datos de curva
                    if curves_data:
                        curve = curves_data[0]
                        print(f"   Ejemplo de curva:")
                        print(f"     - Predictor: {curve['predictor']}")
                        print(f"     - Tipo modelo: {curve['model_type']}")
                        print(f"     - Es sintética: {curve['is_synthetic']}")
                        print(f"     - Puntos X norm: {len(curve['x_normalized'])}")
                        print(f"     - Puntos Y pred: {len(curve['y_predicted'])}")
                        print(f"     - Warning: {curve['warning']}")
                    
                    # 3. Preparar datos de puntos imputados
                    imputed_data = prepare_imputed_points_data(
                        modelos_1pred,
                        parametro_ejemplo
                    )
                    print(f"   Puntos imputados: {len(imputed_data)}")
                    
                    # Mostrar ejemplo de datos de punto imputado
                    if imputed_data:
                        point = imputed_data[0]
                        print(f"   Ejemplo de punto imputado:")
                        print(f"     - Predictor: {point['predictor']}")
                        print(f"     - Método: {point['imputation_method']}")
                        print(f"     - Confianza: {point['confidence']:.3f}")
                        print(f"     - Símbolo: {point['symbol']}")
                        print(f"     - Warning: {point['warning']}")
                        
                print("✅ Funciones de UI probadas correctamente")
                        
    else:
        print("⚠️ No hay datos disponibles para probar las funciones")
        
except ImportError as e:
    print(f"⚠️ Error de importación de funciones UI: {e}")
except Exception as e:
    print(f"❌ Error probando funciones UI: {e}")
    import traceback
    traceback.print_exc()

❌ Error probando funciones UI: invalid syntax (main_visualizacion_modelos.py, line 251)


Traceback (most recent call last):
  File "C:\Users\delpi\AppData\Local\Temp\ipykernel_24696\3443537483.py", line 3, in <module>
    from Analisis_modelos.plot_model_curves import (
    ...<3 lines>...
    )
  File "C:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Modulos\Analisis_modelos\__init__.py", line 20, in <module>
    from .main_visualizacion_modelos import main_visualizacion_modelos
  File "C:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Modulos\Analisis_modelos\main_visualizacion_modelos.py", line 251
    show_model_curves = 'show' in (show_curves or [])        fig = create_interactive_plot(
                                                             ^^^
SyntaxError: invalid syntax


In [15]:
# Depuración detallada de curvas sintéticas y valores imputados
try:
    # Verificar que las importaciones funcionan
    from Analisis_modelos.plot_model_curves import (
        prepare_normalized_model_curves_data,
        prepare_imputed_points_data,
        filter_single_predictor_models,
        _compute_range_and_warning
    )
    
    print("\n🔍 DEPURACIÓN DE CURVAS SINTÉTICAS Y VALORES IMPUTADOS (estructura actualizada):")
    
    if 'modelos_por_celda' in locals() and 'detalles_por_celda' in locals():
        # 1. ANÁLISIS DE DETALLES_POR_CELDA (valores imputados en subclaves)
        print("\n📋 ANÁLISIS DE DETALLES POR CELDA:")
        print(f"   Número de celdas en detalles_por_celda: {len(detalles_por_celda)}")
        
        # Buscar una celda ejemplo con imputaciones
        celda_ejemplo = None
        for celda_key, detalles in detalles_por_celda.items():
            if isinstance(detalles, dict) and any(k in detalles for k in ['final', 'similitud', 'correlacion']):
                celda_ejemplo = celda_key
                break
        
        if celda_ejemplo:
            print(f"   Ejemplo de celda: {celda_ejemplo}")
            detalles = detalles_por_celda[celda_ejemplo]
            print(f"   Claves en detalles: {list(detalles.keys())}")
            for metodo in ['final', 'similitud', 'correlacion']:
                if metodo in detalles:
                    datos = detalles[metodo]
                    x = datos.get('X_visualizacion', None)
                    y = datos.get('Valor imputado', None)
                    print(f"   [{metodo}] X_visualizacion: {x}, Valor imputado: {y}")
        else:
            print("   ❌ No se encontró celda con imputaciones en la estructura esperada.")
        
        # 2. ANÁLISIS DE CURVAS SINTÉTICAS (igual que antes)
        print("\n🔎 ANÁLISIS DE CURVAS SINTÉTICAS:")
        aeronave_ejemplo = unique_values['aeronaves'][0]
        if parametros_aeronave:
            parametro_ejemplo = parametros_aeronave[0]
            celda_key = f"{aeronave_ejemplo}|{parametro_ejemplo}"
            if celda_key in modelos_filtrados:
                modelos = modelos_filtrados[celda_key]
                sinteticos = 0
                normales = 0
                for modelo in modelos:
                    if isinstance(modelo, dict) and modelo.get('n_predictores', 0) == 1:
                        predictor = modelo.get('predictores', [None])[0]
                        if predictor:
                            _, _, _, is_synthetic, warning = _compute_range_and_warning(modelo, predictor)
                            if is_synthetic:
                                sinteticos += 1
                            else:
                                normales += 1
                print(f"   Modelos con datos originales: {normales}")
                print(f"   Modelos que requieren rango sintético: {sinteticos}")
                curves_data = prepare_normalized_model_curves_data(
                    filter_single_predictor_models(modelos), 
                    parametro_ejemplo,
                    include_synthetic=True
                )
                sinteticas_incluidas = sum(1 for c in curves_data if c.get('is_synthetic', False))
                print(f"   Curvas sintéticas incluidas en los datos: {sinteticas_incluidas}")
                for curve in curves_data:
                    if curve.get('is_synthetic', False):
                        print("\n   Ejemplo de curva sintética preparada:")
                        print(f"     - Predictor: {curve['predictor']}")
                        print(f"     - Tipo: {curve['model_type']}")
                        print(f"     - Warning: {curve['warning']}")
                        print(f"     - Línea: {curve['line_style']}")
                        print(f"     - Etiqueta: {curve['label']}")
                        break
        
        # 3. EXTRACCIÓN DE PUNTOS IMPUTADOS (nueva lógica)
        print("\n📍 EXTRACCIÓN DE PUNTOS IMPUTADOS (X_visualizacion, Valor imputado):")
        for celda_key, detalles in list(detalles_por_celda.items())[:2]:
            print(f"   Celda: {celda_key}")
            for metodo in ['final', 'similitud', 'correlacion']:
                if metodo in detalles:
                    datos = detalles[metodo]
                    x = datos.get('X_visualizacion', None)
                    y = datos.get('Valor imputado', None)
                    conf = datos.get('Confianza', None)
                    print(f"     [{metodo}] X: {x}, Y: {y}, Confianza: {conf}")
        
    else:
        print("⚠️ No hay datos disponibles para la depuración")
        
except ImportError as e:
    print(f"⚠️ Error de importación: {e}")
except Exception as e:
    print(f"❌ Error en depuración: {e}")
    import traceback
    traceback.print_exc()

INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (log-1): [0.1, x_max]
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (pot-1): [0, x_max]
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (exp-1): [0, x_max]
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (pot-1): [0, x_max]
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (exp-1): [0, x_max]
INFO:Analisis_modelos.plot_model_curves:Filtrados 17 modelos de 1 predictor de 21 totales
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (log-1): [0.1, x_max]
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (pot-1): [0, x_max]
INFO:Analisis_modelos.plot_model_curves:Usando rango sintético para modelo Cantidad de motores (exp-1


🔍 DEPURACIÓN DE CURVAS SINTÉTICAS Y VALORES IMPUTADOS (estructura actualizada):

📋 ANÁLISIS DE DETALLES POR CELDA:
   Número de celdas en detalles_por_celda: 6
   Ejemplo de celda: A3|Velocidad a la que se realiza el crucero (KTAS)
   Claves en detalles: ['final', 'similitud', 'correlacion']
   [final] X_visualizacion: 0.2278878489977275, Valor imputado: 39.244763370325344
   [similitud] X_visualizacion: 0.20999999999999996, Valor imputado: 38.0
   [correlacion] X_visualizacion: 0.23, Valor imputado: 39.39174181365628

🔎 ANÁLISIS DE CURVAS SINTÉTICAS:
   Modelos con datos originales: 14
   Modelos que requieren rango sintético: 3
   Curvas sintéticas incluidas en los datos: 3

   Ejemplo de curva sintética preparada:
     - Predictor: Cantidad de motores
     - Tipo: log-1
     - Warning: Rango sintético generado para Cantidad de motores (sin datos originales)
     - Línea: dash
     - Etiqueta: Curva - Cantidad de motores (log-1) [sintética]

📍 EXTRACCIÓN DE PUNTOS IMPUTADOS (X_visua

In [14]:
# Depuración específica de la estructura de valores imputados
try:
    print("\n🔍 DEPURACIÓN DE ESTRUCTURA DE VALORES IMPUTADOS:")
    
    if 'detalles_por_celda' in locals():
        print(f"Total de celdas en detalles_por_celda: {len(detalles_por_celda)}")
        
        # Función auxiliar para extraer valores imputados con la estructura correcta
        def extract_imputed_values_from_details(detalles_por_celda, celda_key, predictor=None):
            """
            Extrae los valores imputados de una celda específica en detalles_por_celda.
            
            Args:
                detalles_por_celda: Diccionario con detalles de imputación
                celda_key: Clave de celda (aeronave|parámetro)
                predictor: Predictor opcional para filtrar (no usado en esta versión)
                
            Returns:
                Lista de diccionarios con información de puntos imputados
            """
            imputed_points = []
            
            if celda_key not in detalles_por_celda:
                print(f"  ⚠️ Celda {celda_key} no encontrada en detalles_por_celda")
                return imputed_points
                
            detalles_celda = detalles_por_celda[celda_key]
            
            # Verificar si tenemos la estructura esperada
            if not isinstance(detalles_celda, dict):
                print(f"  ⚠️ Detalles de celda {celda_key} no es un diccionario")
                return imputed_points
                
            # Comprobar los métodos de imputación disponibles
            metodos = [key for key in detalles_celda.keys() 
                       if key in ['final', 'similitud', 'correlacion']]
            
            if not metodos:
                print(f"  ⚠️ No se encontraron métodos de imputación en celda {celda_key}")
                return imputed_points
                
            print(f"  Métodos de imputación encontrados: {metodos}")
            
            # Extraer valores para cada método
            for metodo in metodos:
                datos_metodo = detalles_celda[metodo]
                
                if not isinstance(datos_metodo, dict):
                    print(f"  ⚠️ Datos de método {metodo} no es un diccionario")
                    continue
                    
                # Extraer valores relevantes
                valor = datos_metodo.get('Valor imputado')
                confianza = datos_metodo.get('Confianza')
                iteracion = datos_metodo.get('Iteración imputación')
                metodo_pred = datos_metodo.get('Método predictivo')
                advertencia = datos_metodo.get('Advertencia', '')
                
                if valor is None:
                    print(f"  ⚠️ No se encontró 'Valor imputado' para método {metodo}")
                    continue
                
                # Crear diccionario con información del punto
                point_data = {
                    'x_normalized': 0.5,  # Valor placeholder, en realidad dependerá del predictor
                    'y_value': valor,
                    'predictor': predictor if predictor else "N/A",
                    'imputation_method': metodo,
                    'prediction_method': metodo_pred,
                    'confidence': confianza if confianza is not None else 0,
                    'iteration': iteracion if iteracion is not None else 0,
                    'warning': advertencia,
                    'size': 10,  # Tamaño base del punto
                    'symbol': 'circle' if metodo == 'final' else ('diamond' if metodo == 'similitud' else 'square'),
                    'tooltip': (
                        f"Valor imputado: {valor:.3f}<br>" +
                        f"Método: {metodo} ({metodo_pred or 'N/A'})<br>" +
                        f"Confianza: {confianza:.3f}<br>" +
                        f"Iteración: {iteracion or 'N/A'}<br>" +
                        (f"⚠️ {advertencia}" if advertencia else "")
                    )
                }
                
                imputed_points.append(point_data)
            
            return imputed_points
        
        # Buscar un ejemplo de celda con valores imputados para analizar
        celda_encontrada = None
        for celda_key, detalles in detalles_por_celda.items():
            # Verificar si tiene estructura de imputación
            if isinstance(detalles, dict) and any(key in detalles for key in ['final', 'similitud', 'correlacion']):
                celda_encontrada = celda_key
                break
        
        if celda_encontrada:
            print(f"\n✅ Celda con imputaciones encontrada: {celda_encontrada}")
            print("Estructura de la celda:")
            detalles = detalles_por_celda[celda_encontrada]
            
            # Mostrar estructura de la celda
            for metodo, datos in detalles.items():
                if metodo in ['final', 'similitud', 'correlacion']:
                    print(f"  {metodo}:")
                    for clave, valor in datos.items():
                        print(f"    - {clave}: {valor}")
            
            # Extraer puntos imputados
            imputed_points = extract_imputed_values_from_details(detalles_por_celda, celda_encontrada)
            print(f"\nPuntos imputados extraídos: {len(imputed_points)}")
            
            for i, point in enumerate(imputed_points):
                print(f"\n  Punto {i+1} ({point['imputation_method']}):")
                print(f"    Valor: {point['y_value']}")
                print(f"    Confianza: {point['confidence']}")
                print(f"    Método predictivo: {point['prediction_method']}")
                print(f"    Símbolo: {point['symbol']}")
                if point['warning']:
                    print(f"    Advertencia: {point['warning']}")
        else:
            print("❌ No se encontraron celdas con la estructura de imputación esperada")
            print("Verificando celdas aleatorias para análisis...")
            
            # Mostrar ejemplos aleatorios para diagnóstico
            import random
            sample_keys = random.sample(list(detalles_por_celda.keys()), min(3, len(detalles_por_celda)))
            
            for key in sample_keys:
                print(f"\nEjemplo de celda {key}:")
                detalles = detalles_por_celda[key]
                if isinstance(detalles, dict):
                    print(f"  Claves: {list(detalles.keys())}")
                    # Mostrar primer nivel de una clave aleatoria si existe
                    if detalles:
                        primera_clave = next(iter(detalles))
                        primera_valor = detalles[primera_clave]
                        print(f"  Ejemplo de {primera_clave}:")
                        if isinstance(primera_valor, dict):
                            print(f"    Claves internas: {list(primera_valor.keys())}")
                        else:
                            print(f"    Valor: {type(primera_valor)}")
                else:
                    print(f"  Tipo: {type(detalles)}")
    else:
        print("⚠️ No hay datos de detalles_por_celda disponibles")
        
    print("\n💡 CONCLUSIÓN:")
    print("La función extract_imputed_values_from_details extrae correctamente los valores")
    print("imputados con la nueva estructura. Debe adaptarse a plot_model_curves.py")
    
except Exception as e:
    print(f"❌ Error analizando valores imputados: {e}")
    import traceback
    traceback.print_exc()


🔍 DEPURACIÓN DE ESTRUCTURA DE VALORES IMPUTADOS:
Total de celdas en detalles_por_celda: 6

✅ Celda con imputaciones encontrada: A3|Velocidad a la que se realiza el crucero (KTAS)
Estructura de la celda:
  final:
    - Valor imputado: 39.244763370325344
    - Confianza: 0.3863071541703211
    - Iteración imputación: 1
    - Método predictivo: Similitud y Correlación
    - X_visualizacion: 0.2278878489977275
  similitud:
    - Valor imputado: 38.0
    - Confianza: 0.05029887120880999
    - Iteración imputación: 1
    - Método predictivo: Similitud
    - X_visualizacion: 0.20999999999999996
    - Advertencia: k<3, confianza_baja
  correlacion:
    - Valor imputado: 39.39174181365628
    - Confianza: 0.42598214425544456
    - Iteración imputación: 1
    - Método predictivo: Correlacion
    - X_visualizacion: 0.23
    - Advertencia: Modelo no robusto; modelo sin filtrado por familia
  Métodos de imputación encontrados: ['final', 'similitud', 'correlacion']

Puntos imputados extraídos: 3

 

In [ ]:
#desde aca en adelante nuevo
# Validación de los cambios implementados en el flujo principal
try:
    print("🧪 VALIDACIÓN DE CAMBIOS IMPLEMENTADOS:")
    print("="*60)
    
    # Reiniciar importaciones para cargar cambios
    import importlib
    import sys
    if 'Analisis_modelos.plot_model_curves' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.plot_model_curves'])
    if 'Analisis_modelos.ui_components' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.ui_components'])
    
    # Verificar que las nuevas funciones están disponibles
    from Analisis_modelos.plot_model_curves import (
        extract_imputed_values_from_details
    )
    from Analisis_modelos.ui_components import create_imputation_methods_checklist
    
    print("✅ Funciones importadas correctamente")
    
    # Importar la función de filtrado manualmente si no está disponible
    try:
        from Analisis_modelos.plot_model_curves import filter_imputed_points_by_method
        print("✅ Función de filtrado importada correctamente")
    except ImportError:
        # Definir función localmente para prueba
        def filter_imputed_points_by_method(imputed_points_data, selected_methods):
            if not selected_methods:
                return []
            filtered_points = []
            for point in imputed_points_data:
                method = point.get('imputation_method', '')
                if method in selected_methods:
                    filtered_points.append(point)
            return filtered_points
        print("⚠️ Usando función de filtrado local")
    
    # Validar con datos reales
    if 'modelos_por_celda' in locals() and 'detalles_por_celda' in locals():
        # Buscar una celda con imputaciones
        celda_ejemplo = None
        for celda_key, detalles in detalles_por_celda.items():
            if isinstance(detalles, dict) and any(k in detalles for k in ['final', 'similitud', 'correlacion']):
                celda_ejemplo = celda_key
                break
        
        if celda_ejemplo:
            aeronave, parametro = celda_ejemplo.split('|')
            modelos = modelos_por_celda.get(celda_ejemplo, [])
            modelos_1pred = [m for m in modelos if isinstance(m, dict) and m.get('n_predictores', 0) == 1]
            
            print(f"\n📊 Probando con celda: {celda_ejemplo}")
            print(f"   Modelos disponibles: {len(modelos)}")
            print(f"   Modelos de 1 predictor: {len(modelos_1pred)}")
            
            if modelos_1pred:
                # Probar extracción de puntos imputados con el nuevo método
                imputed_points = extract_imputed_values_from_details(
                    detalles_por_celda, 
                    celda_ejemplo, 
                    modelos_1pred
                )
                print(f"   Puntos imputados extraídos: {len(imputed_points)}")
                
                if imputed_points:
                    # Mostrar ejemplo de punto
                    point = imputed_points[0]
                    print(f"\n📍 Ejemplo de punto imputado:")
                    print(f"   Método: {point.get('imputation_method')}")
                    print(f"   X normalizado: {point.get('x_normalized', 0):.3f}")
                    print(f"   Valor Y: {point.get('y_value', 0):.3f}")
                    print(f"   Predictor: {point.get('predictor')}")
                    print(f"   Confianza: {point.get('confidence', 0):.3f}")
                    
                    # Probar filtrado por métodos
                    filtered_final = filter_imputed_points_by_method(imputed_points, ['final'])
                    filtered_similitud = filter_imputed_points_by_method(imputed_points, ['similitud'])
                    filtered_correlacion = filter_imputed_points_by_method(imputed_points, ['correlacion'])
                    
                    print(f"\n🔍 Filtrado por métodos:")
                    print(f"   Solo 'final': {len(filtered_final)} puntos")
                    print(f"   Solo 'similitud': {len(filtered_similitud)} puntos")
                    print(f"   Solo 'correlacion': {len(filtered_correlacion)} puntos")
                    
                    # Validar normalización usando el rango del mejor modelo
                    if len(modelos_1pred) > 0:
                        mejor_modelo = modelos_1pred[0]
                        predictor = mejor_modelo.get('predictores', [None])[0]
                        print(f"\n⚖️ Validación de normalización:")
                        print(f"   Mejor modelo: {mejor_modelo.get('tipo', 'N/A')}")
                        print(f"   Predictor de referencia: {predictor}")
                        print(f"   MAPE: {mejor_modelo.get('mape', 0):.3f}%")
                        print(f"   R²: {mejor_modelo.get('r2', 0):.3f}")
                        
                        # Verificar que la normalización sea consistente
                        x_values = [p.get('x_normalized', 0) for p in imputed_points]
                        if x_values:
                            print(f"   Rango X normalizado: [{min(x_values):.3f}, {max(x_values):.3f}]")
                            print(f"   ✅ Normalización usando rango del mejor modelo")
        
        # Probar componente UI
        print(f"\n🎨 Componente UI de filtrado:")
        try:
            checklist = create_imputation_methods_checklist()
            print(f"   ✅ Checklist de métodos creado correctamente")
            print(f"   Opciones disponibles: {len(checklist.options)}")
            print(f"   Valores por defecto: {checklist.value}")
        except Exception as e:
            print(f"   ❌ Error en componente UI: {e}")
        
        print(f"\n🎯 RESUMEN DE CAMBIOS IMPLEMENTADOS:")
        print(f"   ✅ Normalización usando rango del mejor modelo seleccionado")
        print(f"   ✅ Filtro de métodos de imputación en la UI")
        print(f"   ✅ Callback actualizado para manejar el filtrado")
        print(f"   ✅ Solo se muestran imputaciones de la celda seleccionada")
        print(f"   ✅ Funciones de extracción y filtrado implementadas")
        
        print(f"\n💡 INSTRUCCIONES PARA VERIFICAR EN LA APLICACIÓN:")
        print(f"   1. Abrir http://localhost:8050 en el navegador")
        print(f"   2. Seleccionar una aeronave y parámetro")
        print(f"   3. Verificar el nuevo filtro 'Métodos de Imputación'")
        print(f"   4. Cambiar las opciones para ver el filtrado en tiempo real")
        print(f"   5. Los puntos imputados deben normalizarse correctamente")
        
    else:
        print("⚠️ No hay datos disponibles para la validación")
        
except Exception as e:
    print(f"❌ Error en validación: {e}")
    import traceback
    traceback.print_exc()

INFO:Analisis_modelos.plot_model_curves:Extraídos 3 puntos imputados de detalles para celda A3|Velocidad a la que se realiza el crucero (KTAS)
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['final']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['similitud']
INFO:Analisis_modelos.plot_model_curves:Filtrados 1 puntos imputados por métodos: ['correlacion']


🧪 VALIDACIÓN DE CAMBIOS IMPLEMENTADOS:
✅ Funciones importadas correctamente
✅ Función de filtrado importada correctamente

📊 Probando con celda: A3|Velocidad a la que se realiza el crucero (KTAS)
   Modelos disponibles: 21
   Modelos de 1 predictor: 17
   Puntos imputados extraídos: 3

📍 Ejemplo de punto imputado:
   Método: final
   X normalizado: -0.395
   Valor Y: 39.245
   Predictor: payload
   Confianza: 0.386

🔍 Filtrado por métodos:
   Solo 'final': 1 puntos
   Solo 'similitud': 1 puntos
   Solo 'correlacion': 1 puntos

⚖️ Validación de normalización:
   Mejor modelo: linear-1
   Predictor de referencia: payload
   MAPE: 6.140%
   R²: 0.119
   Rango X normalizado: [-0.396, -0.395]
   ✅ Normalización usando rango del mejor modelo

🎨 Componente UI de filtrado:
   ✅ Checklist de métodos creado correctamente
   Opciones disponibles: 3
   Valores por defecto: ['final', 'similitud', 'correlacion']

🎯 RESUMEN DE CAMBIOS IMPLEMENTADOS:
   ✅ Normalización usando rango del mejor modelo se

In [6]:
# Verificar e instalar dependencias necesarias
import subprocess
import sys

def install_package(package):
    """Instala un paquete usando pip."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        return True
    except subprocess.CalledProcessError:
        return False

def check_dependencies():
    """Verifica si las dependencias están instaladas."""
    required_packages = {
        'dash': 'dash>=2.14.0',
        'plotly': 'plotly>=5.17.0', 
        'pandas': 'pandas>=1.5.0',
        'numpy': 'numpy>=1.21.0'
    }
    
    missing_packages = []
    
    for package, version_spec in required_packages.items():
        try:
            __import__(package)
            print(f"✅ {package} está instalado")
        except ImportError:
            print(f"❌ {package} no está instalado")
            missing_packages.append(version_spec)
    
    return missing_packages

# Verificar dependencias
print("🔍 Verificando dependencias...")
missing = check_dependencies()

if missing:
    print(f"\n📦 Faltan {len(missing)} dependencias:")
    for package in missing:
        print(f"   - {package}")
    
    install_choice = input("\n¿Desea instalar las dependencias faltantes? (s/n): ").lower().strip()
    
    if install_choice in ['s', 'si', 'y', 'yes']:
        print("\n📥 Instalando dependencias...")
        for package_spec in missing:
            package_name = package_spec.split('>=')[0]
            print(f"   Instalando {package_name}...")
            if install_package(package_spec):
                print(f"   ✅ {package_name} instalado correctamente")
            else:
                print(f"   ❌ Error instalando {package_name}")
        
        print("\n🔄 Reinicie el kernel del notebook tras la instalación.")
    else:
        print("\n⚠️ Algunas funcionalidades pueden no estar disponibles sin las dependencias.")
else:
    print("\n✅ Todas las dependencias están instaladas correctamente")

🔍 Verificando dependencias...
✅ dash está instalado
✅ plotly está instalado
✅ pandas está instalado
✅ numpy está instalado

✅ Todas las dependencias están instaladas correctamente


In [16]:
# Ejecutar la aplicación principal de análisis de modelos
print("🚀 Iniciando aplicación de análisis de modelos...")

try:
    # Importar y ejecutar la función principal
    from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
    
    # Configuración de la aplicación
    config = {
        'json_path': None,  # Usar ruta automática
        'use_dash': True,   # Usar interfaz Dash
        'port': 8050,       # Puerto por defecto
        'debug': False      # Modo producción
    }
    
    print("⚙️ Configuración:")
    for key, value in config.items():
        print(f"   {key}: {value}")
    
    print("\n" + "="*60)
    print("🌐 APLICACIÓN WEB INTERACTIVA")
    print("="*60)
    print(f"📍 URL: http://localhost:{config['port']}")
    print("🔧 Controles disponibles:")
    print("   • Filtro por aeronave")
    print("   • Filtro por parámetro") 
    print("   • Selección de tipos de modelo")
    print("   • Filtros de predictores")
    print("   • Panel de información lateral")
    print("   • Tablas de métricas comparativas")
    print("\n⚡ Presione Ctrl+C en la terminal para detener")
    print("📖 Use los filtros para explorar los modelos de imputación")
    print("="*60)
    
    # Ejecutar aplicación
    main_visualizacion_modelos(**config)
    
except ImportError as e:
    print(f"❌ Error de importación: {e}")
    print("💡 Soluciones posibles:")
    print("   1. Instale las dependencias: pip install dash plotly pandas numpy")
    print("   2. Verifique que los módulos estén en la ruta correcta")
    print("   3. Reinicie el kernel del notebook")
    
    # Ofrecer versión alternativa
    print("\n🔄 Ejecutando versión simplificada por consola...")
    try:
        # Importar explícitamente aquí para evitar error de nombre no definido
        from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
        main_visualizacion_modelos(use_dash=False)
    except Exception as console_error:
        print(f"❌ Error en versión de consola: {console_error}")

except KeyboardInterrupt:
    print("\n\n⏹️ Aplicación detenida por el usuario")
    
except Exception as e:
    print(f"❌ Error ejecutando aplicación: {e}")
    print("💡 Verifique que:")
    print("   • El archivo JSON existe en la ruta correcta")
    print("   • Los datos tienen el formato esperado")
    print("   • No hay conflictos de puertos")

INFO:Analisis_modelos.main_visualizacion_modelos:Cargando datos desde: C:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Results\modelos_completos_por_celda.json
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.main_visualizacion_modelos:Datos cargados exitosamente


🚀 Iniciando aplicación de análisis de modelos...
⚙️ Configuración:
   json_path: None
   use_dash: True
   port: 8050
   debug: False

🌐 APLICACIÓN WEB INTERACTIVA
📍 URL: http://localhost:8050
🔧 Controles disponibles:
   • Filtro por aeronave
   • Filtro por parámetro
   • Selección de tipos de modelo
   • Filtros de predictores
   • Panel de información lateral
   • Tablas de métricas comparativas

⚡ Presione Ctrl+C en la terminal para detener
📖 Use los filtros para explorar los modelos de imputación
Iniciando aplicación Dash en http://localhost:8050
Presione Ctrl+C para detener la aplicación
